# CRIO Timestamp issue

Craig Lage - 03-Sep-24

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler

import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.utils import dayObsIntToString
from astro_metadata_translator import ObservationInfo

from lsst_efd_client import EfdClient
from lsst_efd_client import merge_packed_time_series as mpts

In [ ]:
def _getEfdData(client, dataSeries, startTime, endTime):
    """A synchronous wrapper for geting the data from the EFD.

    This exists so that the top level functions don't all have to be async def.
    """
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(client.select_time_series(dataSeries, ['*'], startTime.utc, endTime.utc))

In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all"])
client = EfdClient('summit_efd')

In [ ]:
for expId in [2024090500181]:#range(2024090500001, 2024090500008):
    #expId = 2024082800259
    dataId = {'detector':0, 'exposure':expId}
    expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
    
    tStart = expRecord.timespan.begin.tai.to_value("isot")
    tEnd = expRecord.timespan.end.tai.to_value("isot")
    t_start = Time(tStart, scale='tai')
    t_end = Time(tEnd, scale='tai')
    print(t_start, t_end)
    
    mount_position = _getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", t_start, t_end)
    
    az = mpts(mount_position, 'azimuthCalculatedAngle')
    el = mpts(mount_position, 'elevationCalculatedAngle')
    cRIO_ts = mount_position["cRIO_timestamp"]
    timestamps = cRIO_ts.values
    deltaTs = []
    for n in range(1, len(timestamps)):
        deltaTs.append(timestamps[n] - timestamps[n-1])
    
    try:
        torques = _getEfdData(client, "lsst.sal.ATMCS.measuredTorque", t_start, t_end)
        az_torque_1 = mpts(torques, "azimuthMotor1Torque")
        torque_cRIO_ts = torques["cRIO_timestamp"]
        torque_timestamps = cRIO_ts.values
        torque_deltaTs = []
        for n in range(1, len(torque_timestamps)):
            torque_deltaTs.append(torque_timestamps[n] - torque_timestamps[n-1])
            torque_sig = np.std(torque_deltaTs)
    except:
        torque_deltaTs = [0,-1]
        torque_sig = 999
    print(len(timestamps), len(torque_timestamps), len(torque_deltaTs), torque_sig)
    sig = np.std(deltaTs)
    
    %matplotlib inline
    fig = plt.figure(figsize=(8,5))
    plt.subplots_adjust(wspace=1.0)
    plt.subplot(1,2,1)
    plt.title(f"Position cRIO timestamps deltas \n{expId}")
    plt.plot(deltaTs, marker='x')
    plt.text(2.0, 2.2,f"DeltaT sigma = {sig:.2g}")
    plt.xlabel("Time(sec)")
    plt.ylabel("DeltaT between successive timestamps")
    plt.ylim(0, 2.5)
    plt.subplot(1,2,2)
    plt.title(f"Torque cRIO timestamps deltas \n{expId}")
    plt.plot(torque_deltaTs, marker='x')
    if torque_sig == 999:
        plt.text(0.2, 2.2,f"DeltaT is BAD")
    else:    
        plt.text(2.0, 2.2,f"DeltaT sigma = {sig:.2g}")
    plt.xlabel("Time(sec)")
    plt.ylabel("DeltaT between successive timestamps")
    plt.ylim(0, 2.5)
plt.savefig(f"/home/cslage/DATA/cRIO_Timestamps_{expId}.png")

In [ ]:
torques = _getEfdData(client, "lsst.sal.ATMCS.measuredTorque", t_start, t_end)
len(torques)

In [ ]:
len(mount_position)